**Importiere die notwendigen Packages**

In [1]:
# Empfehlung für Colab: Installiere alles was Du brauchst
# %pip install pandas pivottablejs numpy altair matplotlib geopandas folium

In [7]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [8]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bau_umbau_leerkuendigung_whg_baujahr_personen_OD5051

(int_dwh)

In [9]:
# Filepath
fp = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bau_umbau_leerkuendigung_whg_baujahr_personen_OD5051\BAU505OD5051.csv"
#\\szh.loc\ssz\applikationen\OGD_Dropzone\INT_DWH\bau_best_whg_zizahl_wfl_bauperi_ea_quartier_jahr_OD5081\BAU508OD5081.csv


# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )

In der Folge ein paar erste Tests:

In [10]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [11]:
data2betested.head(30000)

,Jahr,Bauperiode,Leerkuendigung,AnzWhg,AnzPers
0,2010,1893-1930,Ja,93,110
1,2010,1893-1930,Nein,113,200
2,2010,1931-1950,Ja,84,94
3,2010,1931-1950,Nein,160,277
4,2010,1951-1960,Ja,57,56
...,...,...,...,...,...
137,2019,1961-1970,Nein,266,388
138,2019,1971-1980,Nein,51,47
139,2019,1981-1990,Nein,133,190
140,2019,1991-2000,Nein,20,39


In [12]:
data2betested.dtypes

Jahr               int64
Bauperiode        object
Leerkuendigung    object
AnzWhg             int64
AnzPers            int64
dtype: object

In [13]:
data2betested.shape

(142, 5)

Beschreibe einzelne Attribute

In [14]:
data2betested.describe()

,Jahr,AnzWhg,AnzPers
count,142,142,142
mean,2014,96,136
std,3,75,117
min,2010,4,0
25%,2012,35,44
50%,2014,79,96
75%,2017,142,203
max,2019,370,520


Wie viele Nullwerte gibt es im Datensatz?

In [15]:
data2betested.isnull().sum()

Jahr              0
Bauperiode        0
Leerkuendigung    0
AnzWhg            0
AnzPers           0
dtype: int64

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [18]:
data2betested['Jahr']=pd.to_datetime(data2betested['Jahr'], format='%Y')
#data2betested['Jahr']=pd.to_datetime(data2betested['Jahr'], format='%Y')
#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested


,Jahr,Bauperiode,Leerkuendigung,AnzWhg,AnzPers
0,2010-01-01,1893-1930,Ja,93,110
1,2010-01-01,1893-1930,Nein,113,200
2,2010-01-01,1931-1950,Ja,84,94
3,2010-01-01,1931-1950,Nein,160,277
4,2010-01-01,1951-1960,Ja,57,56
...,...,...,...,...,...
137,2019-01-01,1961-1970,Nein,266,388
138,2019-01-01,1971-1980,Nein,51,47
139,2019-01-01,1981-1990,Nein,133,190
140,2019-01-01,1991-2000,Nein,20,39


Bereite die Daten für die Grafik unten vor

In [20]:
Jahre = data2betested.groupby("Jahr")
len(Jahre)

10

In [61]:
for key, group in Jahre:
    print(key, len(group))

2010-01-01 00:00:00 15
2011-01-01 00:00:00 14
2012-01-01 00:00:00 15
2013-01-01 00:00:00 14
2014-01-01 00:00:00 14
2015-01-01 00:00:00 13
2016-01-01 00:00:00 16
2017-01-01 00:00:00 15
2018-01-01 00:00:00 14
2019-01-01 00:00:00 12


In [62]:
myJahr = "2019-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr.head(3)


,Jahr,Bauperiode,Leerkuendigung,AnzWhg,AnzPers
130,2019-01-01,1893-1930,Ja,175,296
131,2019-01-01,1893-1930,Nein,144,195
132,2019-01-01,1931-1950,Ja,63,75


**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [65]:
   
test = data2betested.groupby(
   ['Jahr', 'Bauperiode', 'Leerkuendigung']
).agg(
    {
         'AnzWhg':(sum),     # Sum --> (sum)
         'AnzPers': (sum),   
         #'Wohnungsfl': 'max'  
    }
)
test.head(10)


AnzWhg  AnzPers
Jahr       Bauperiode Leerkuendigung                 
2010-01-01 1893-1930  Ja                  93      110
                      Nein               113      200
           1931-1950  Ja                  84       94
                      Nein               160      277
           1951-1960  Ja                  57       56
                      Nein                66       92
           1961-1970  Ja                  26       53
                      Nein                33       47
           1971-1980  Ja                  47        0
                      Nein                57      131

In [64]:
#In den Rohdaten gibt es viele Werte, wo MietwohnungSort leer ist. Was geschieht damit in der Aggregation?      
test2= data2betested[['Jahr', 'Bauperiode', 'Leerkuendigung','AnzWhg']]
test2= data2betested.groupby(['Jahr', 'Leerkuendigung'])[['AnzWhg']].sum()  
test2.head(100000)

AnzWhg
Jahr       Leerkuendigung        
2010-01-01 Ja                 334
           Nein               675
2011-01-01 Ja                 335
           Nein               873
2012-01-01 Ja                 495
           Nein              1466
2013-01-01 Ja                 321
           Nein               745
2014-01-01 Ja                 565
           Nein               904
2015-01-01 Ja                 235
           Nein               761
2016-01-01 Ja                 652
           Nein              1068
2017-01-01 Ja                 675
           Nein              1059
2018-01-01 Ja                 457
           Nein               799
2019-01-01 Ja                 366
           Nein               859

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [53]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [54]:
#mySelection = aktuellesJahr[(data2betested.QuarSort=31)]
mySelection = aktuellesJahr.groupby(['Jahr', 'Bauperiode', 'Leerkuendigung'])['AnzWhg'].sum()
mySelection.head()




Jahr        Bauperiode  Leerkuendigung
2019-01-01  1893-1930   Ja                175
                        Nein              144
            1931-1950   Ja                 63
                        Nein              159
            1951-1960   Ja                 69
Name: AnzWhg, dtype: int64

In [55]:
mySelection = aktuellesJahr[['Jahr', 'Bauperiode', 'Leerkuendigung',"AnzWhg", "AnzPers"]]
mySelection

,Jahr,Bauperiode,Leerkuendigung,AnzWhg,AnzPers
130,2019-01-01,1893-1930,Ja,175,296
131,2019-01-01,1893-1930,Nein,144,195
132,2019-01-01,1931-1950,Ja,63,75
133,2019-01-01,1931-1950,Nein,159,204
134,2019-01-01,1951-1960,Ja,69,58
135,2019-01-01,1951-1960,Nein,65,101
136,2019-01-01,1961-1970,Ja,59,85
137,2019-01-01,1961-1970,Nein,266,388
138,2019-01-01,1971-1980,Nein,51,47
139,2019-01-01,1981-1990,Nein,133,190


In [56]:
agg_mySelection = mySelection.groupby(
   ['Jahr', 'Bauperiode', 'Leerkuendigung']
).agg(
    {
         'AnzWhg':(sum,min, max),     # Sum --> (sum)
         'AnzPers': (sum, min, max)
    }
)
agg_mySelection

AnzWhg           AnzPers          
                                        sum  min  max     sum  min  max
Jahr       Bauperiode Leerkuendigung                                   
2019-01-01 1893-1930  Ja                175  175  175     296  296  296
                      Nein              144  144  144     195  195  195
           1931-1950  Ja                 63   63   63      75   75   75
                      Nein              159  159  159     204  204  204
           1951-1960  Ja                 69   69   69      58   58   58
                      Nein               65   65   65     101  101  101
           1961-1970  Ja                 59   59   59      85   85   85
                      Nein              266  266  266     388  388  388
           1971-1980  Nein               51   51   51      47   47   47
           1981-1990  Nein              133  133  133     190  190  190
           1991-2000  Nein               20   20   20      39   39   39
           Vor 1893   Nein               21   21   21      30   30   30

In [57]:
mySelection = aktuellesJahr[(aktuellesJahr.Bauperiode !="1991-2000") & (aktuellesJahr.Leerkuendigung =="Ja")]

mySelection.head()

,Jahr,Bauperiode,Leerkuendigung,AnzWhg,AnzPers
130,2019-01-01,1893-1930,Ja,175,296
132,2019-01-01,1931-1950,Ja,63,75
134,2019-01-01,1951-1960,Ja,69,58
136,2019-01-01,1961-1970,Ja,59,85


In [58]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]

highlight = alt.selection(type='single', on='mouseover',
                          fields=['Bauperiode'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(mySelection).encode(
    x='AnzPers',
    y='AnzWhg',
    color='Bauperiode'
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

alt.LayerChart(...)

In [59]:
alt.Chart(mySelection).mark_circle(size=60).encode(
    x='AnzPers',
    y='Bauperiode',
    #color='AnzWhg',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='dark2')),
    color=alt.Color('AnzPers', scale=alt.Scale(scheme='greens')),
    tooltip=['Jahr','Bauperiode','AnzWhg', 'AnzPers']
).interactive() # this makes the axes interactive: now you can zoom & pan




alt.Chart(...)